In [21]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [22]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [23]:
subject = 'Amapá - Consumo de Cimento (t)'
split_index = 191 #Referente aos 230 anos de input  
train_split = split_index + 1 -30

In [24]:
data = pd.read_csv('2003_mo_model_input_AP.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data['Unnamed: 0'] = data['Unnamed: 0'].str[5:].astype(float)
data

,Unnamed: 0,Amapá - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Amapá - PIB - Estadual,Amapá - PIB - Construção Civil,Amapá - PIB - Per Capita,Amapá - PIB - Preços de Mercado,Amapá - Desemprego,Amapá - Consumo de Cimento (t)
0,1.0,0.711421,0.724032,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,8.035813e+06,356591.003430,10.883143,7.033593e+06,8.514392,10.392
1,2.0,0.711553,0.690297,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,8.046432e+06,356647.711004,10.885206,7.037356e+06,8.508753,6.857
2,3.0,0.711685,0.669681,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,8.057052e+06,356704.418579,10.887268,7.041120e+06,8.503114,7.011
3,4.0,0.711817,0.660494,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,8.067671e+06,356761.126153,10.889331,7.044883e+06,8.497475,7.122
4,5.0,0.711949,0.648337,9.959690,1.736072,0.917493,1.779329e+08,7.352713e+06,0.951683,8.078290e+06,356817.833728,10.891394,7.048646e+06,8.491835,5.267
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.208
236,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.476
237,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.236
238,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.549


In [25]:
input_data = data.iloc[:split_index + 1,0:-1]
mean = np.mean(input_data, axis=0)
stddev =  np.std(input_data, axis=0)
input_data = ((input_data - mean) /stddev)
input_data

,Unnamed: 0,Amapá - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Amapá - PIB - Estadual,Amapá - PIB - Construção Civil,Amapá - PIB - Per Capita,Amapá - PIB - Preços de Mercado,Amapá - Desemprego
0,-1.593255,-2.943242,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-1.716825,-0.487709,-0.246236,-1.984303,-0.890357
1,-1.303572,-2.870364,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-1.697794,-0.479493,-0.208006,-1.951397,-0.891922
2,-1.013890,-2.797485,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-1.678763,-0.471277,-0.169777,-1.918491,-0.893486
3,-0.724207,-2.724606,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-1.659732,-0.463061,-0.131548,-1.885585,-0.895051
4,-0.434524,-2.651728,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-1.640701,-0.454845,-0.093319,-1.852679,-0.896616
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,0.434524,-0.638974,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,1.145925,-1.614044,-1.566534,1.010496,1.085785
188,0.724207,-0.739823,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,1.135411,-1.605628,-1.579697,0.995294,1.078764
189,1.013890,-0.840672,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,1.124897,-1.597213,-1.592859,0.980093,1.071742
190,1.303572,-0.941521,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,1.114383,-1.588797,-1.606022,0.964891,1.064721


In [26]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-12)
target_data

0      10.000
1       6.136
2       5.122
3       3.926
4       6.197
        ...  
235       NaN
236       NaN
237       NaN
238       NaN
239       NaN
Name: Amapá - Consumo de Cimento (t), Length: 240, dtype: float64

In [27]:
# input para treinamento
train_input = input_data.iloc[:train_split]
train_input

,Unnamed: 0,Amapá - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Amapá - PIB - Estadual,Amapá - PIB - Construção Civil,Amapá - PIB - Per Capita,Amapá - PIB - Preços de Mercado,Amapá - Desemprego
0,-1.593255,-2.943242,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-1.716825,-0.487709,-0.246236,-1.984303,-0.890357
1,-1.303572,-2.870364,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-1.697794,-0.479493,-0.208006,-1.951397,-0.891922
2,-1.013890,-2.797485,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-1.678763,-0.471277,-0.169777,-1.918491,-0.893486
3,-0.724207,-2.724606,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-1.659732,-0.463061,-0.131548,-1.885585,-0.895051
4,-0.434524,-2.651728,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-1.640701,-0.454845,-0.093319,-1.852679,-0.896616
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,-1.303572,1.388995,-0.214006,-0.607704,0.059280,-1.233012,1.031384,0.819304,-0.883659,1.179199,-1.457028,-1.470096,1.047494,1.300799
158,-1.013890,1.332036,-0.434717,-0.620523,0.222389,-1.299304,1.042716,0.808136,-0.950771,1.184540,-1.486109,-1.464204,1.056372,1.292722
159,-0.724207,1.275077,-0.524091,-0.631530,0.191929,-1.248662,1.054049,0.796969,-1.028465,1.189880,-1.515190,-1.458311,1.065250,1.284645
160,-0.434524,1.218118,-0.614500,-0.640320,0.385687,-1.068274,1.065381,0.785801,-1.103668,1.195221,-1.544271,-1.452419,1.074128,1.276568


In [28]:
# Alvo para treinamento
train_target = target_data.iloc[:train_split]
train_target

0      10.000
1       6.136
2       5.122
3       3.926
4       6.197
        ...  
157     8.322
158    11.373
159     9.651
160    10.855
161    13.970
Name: Amapá - Consumo de Cimento (t), Length: 162, dtype: float64

In [29]:
#input de test (Ano 2021)
test_input = input_data.iloc[train_split:]
test_input

,Unnamed: 0,Amapá - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Amapá - PIB - Estadual,Amapá - PIB - Construção Civil,Amapá - PIB - Per Capita,Amapá - PIB - Preços de Mercado,Amapá - Desemprego
162,0.144841,1.104200,-0.601510,-0.653307,0.157879,-1.255326,1.088045,0.763466,-1.213929,1.205903,-1.602434,-1.440634,1.091884,1.260415
163,0.434524,1.047242,-0.786068,-0.653814,0.548855,-1.118679,1.099377,0.752299,-1.292173,1.211243,-1.631515,-1.434741,1.100762,1.252338
164,0.724207,0.990283,-0.830387,-0.645656,0.385685,-0.936336,1.110709,0.741131,-1.324219,1.216584,-1.660596,-1.428849,1.109639,1.244261
165,1.013890,0.933324,-0.801089,-0.634274,0.420965,-0.931790,1.122042,0.729964,-1.344446,1.221925,-1.689677,-1.422956,1.118517,1.236184
166,1.303572,0.876365,-0.959917,-0.624035,0.548870,-1.168522,1.133374,0.718796,-1.381638,1.227266,-1.718758,-1.417063,1.127395,1.228107
167,1.593255,0.819406,-1.022309,-0.619791,0.222499,-1.285611,1.144706,0.707629,-1.411208,1.232606,-1.747839,-1.411171,1.136273,1.220031
168,-1.593255,0.762447,-1.074401,-0.620112,-0.614754,-1.446316,1.156038,0.696461,-1.412953,1.237947,-1.776921,-1.405278,1.145151,1.211954
169,-1.303572,0.704491,-1.119597,-0.615973,-0.478229,-1.357832,1.167287,0.681823,-1.491464,1.236412,-1.768257,-1.411038,1.142798,1.205535
170,-1.013890,0.646535,-1.078648,-0.609777,-0.393157,-1.235682,1.178536,0.667184,-1.573805,1.234876,-1.759592,-1.416798,1.140444,1.199117
171,-0.724207,0.588578,-1.055426,-0.602141,-0.557552,-1.065340,1.189784,0.652545,-1.564950,1.233341,-1.750928,-1.422558,1.138091,1.192699


In [30]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[train_split:split_index + 1]
test_target

162    12.399
163    11.268
164     8.741
165    12.358
166    11.876
167    11.713
168    11.449
169     7.805
170     7.726
171     8.516
172     8.503
173    10.175
174    11.775
175     9.724
176    10.069
177    13.930
178    11.918
179    11.757
180     9.691
181     7.378
182     6.970
183     7.612
184     8.442
185     8.835
186    11.251
187    11.342
188    13.110
189    13.195
190    10.053
191    12.274
Name: Amapá - Consumo de Cimento (t), dtype: float64

In [31]:
def validation_splitter(df, div_factor):
    split_factor = len(df)//div_factor
    positions_to_drop = []
    for i in range(split_factor):
        pos = df.shape[0] - (i*div_factor + 1)
        positions_to_drop.append(pos)
    
    return df.drop(positions_to_drop), df.iloc[positions_to_drop]

In [32]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
#     train, train_val = validation_splitter(train_input, 5)
#     target,target_val = validation_splitter(train_target, 5)
#     display(train)
#     display(train_val)
#     display(target)
#     display(target_val)
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                        train_target, 
                        epochs=10000,
                        validation_split=0.1,
                        callbacks=[early_stopping],
                        verbose=want_verbose)
    return model, history

In [33]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(25)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [34]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[3218670190, 1437645356, 2303635714, 1251381389, 3940570480, 789749933, 1755794941, 3063943631, 3671747053, 2798696150, 2397337414, 1851280778, 2696736403, 2651770250, 1717574788, 250892912, 1120617525, 3905137210, 4057865595, 2380847272, 2968047306, 2329841915, 4284486938, 3330446455, 3980462693]


Step: 0 ___________________________________________
val_loss: 4.552715301513672
winner_seed: 3218670190


Step: 1 ___________________________________________
val_loss: 3.2802815437316895
winner_seed: 1437645356


Step: 2 ___________________________________________
val_loss: 17.710159301757812


Step: 3 ___________________________________________
val_loss: 3.8951847553253174


Step: 4 ___________________________________________
val_loss: 4.236536979675293


Step: 5 ___________________________________________
val_loss: 3.340890407562256


Step: 6 ___________________________________________
val_loss: 51.453147888183594


Step: 7 ___________________________________________
val_loss: 3.162452697

In [35]:
trained_model, history = neural_network_model(train_input, 
                                              train_target, 
                                              want_verbose=1, 
                                              seed=winner_seed)

Epoch 1/10000
5/5 [==============================] - 1s 25ms/step - loss: 102.0666 - val_loss: 877.5972
Epoch 2/10000
5/5 [==============================] - 0s 6ms/step - loss: 48.2633 - val_loss: 112.0770
Epoch 3/10000
5/5 [==============================] - 0s 6ms/step - loss: 18.9159 - val_loss: 161.5453
Epoch 4/10000
5/5 [==============================] - 0s 6ms/step - loss: 16.2686 - val_loss: 126.7643
Epoch 5/10000
5/5 [==============================] - 0s 7ms/step - loss: 14.6316 - val_loss: 91.2276
Epoch 6/10000
5/5 [==============================] - 0s 6ms/step - loss: 10.1372 - val_loss: 93.0454
Epoch 7/10000
5/5 [==============================] - 0s 6ms/step - loss: 9.9218 - val_loss: 97.8099
Epoch 8/10000
5/5 [==============================] - 0s 6ms/step - loss: 10.1933 - val_loss: 141.8658
Epoch 9/10000
5/5 [==============================] - 0s 7ms/step - loss: 10.5047 - val_loss: 147.7420
Epoch 10/10000
5/5 [==============================] - 0s 7ms/step - loss: 8.6961 - v

5/5 [==============================] - 0s 5ms/step - loss: 5.7157 - val_loss: 67.9485
Epoch 162/10000
5/5 [==============================] - 0s 6ms/step - loss: 5.4192 - val_loss: 73.7561
Epoch 163/10000
5/5 [==============================] - 0s 6ms/step - loss: 5.3152 - val_loss: 87.3635
Epoch 164/10000
5/5 [==============================] - 0s 6ms/step - loss: 5.8317 - val_loss: 50.0908
Epoch 165/10000
5/5 [==============================] - 0s 6ms/step - loss: 5.2975 - val_loss: 61.3773
Epoch 166/10000
5/5 [==============================] - 0s 6ms/step - loss: 5.0529 - val_loss: 59.6463
Epoch 167/10000
5/5 [==============================] - 0s 6ms/step - loss: 5.3674 - val_loss: 57.4173
Epoch 168/10000
5/5 [==============================] - 0s 6ms/step - loss: 5.3439 - val_loss: 81.0288
Epoch 169/10000
5/5 [==============================] - 0s 6ms/step - loss: 5.0234 - val_loss: 65.6255
Epoch 170/10000
5/5 [==============================] - 0s 6ms/step - loss: 5.2354 - val_loss: 39.4

5/5 [==============================] - 0s 7ms/step - loss: 4.0372 - val_loss: 4.7153
Epoch 323/10000
5/5 [==============================] - 0s 6ms/step - loss: 3.5868 - val_loss: 4.5411
Epoch 324/10000
5/5 [==============================] - 0s 6ms/step - loss: 3.7357 - val_loss: 4.7760
Epoch 325/10000
5/5 [==============================] - 0s 6ms/step - loss: 3.6474 - val_loss: 9.0031
Epoch 326/10000
5/5 [==============================] - 0s 6ms/step - loss: 4.0258 - val_loss: 5.0046
Epoch 327/10000
5/5 [==============================] - 0s 6ms/step - loss: 4.1501 - val_loss: 5.5459
Epoch 328/10000
5/5 [==============================] - 0s 6ms/step - loss: 3.5190 - val_loss: 5.0495
Epoch 329/10000
5/5 [==============================] - 0s 6ms/step - loss: 3.5626 - val_loss: 3.7257
Epoch 330/10000
5/5 [==============================] - 0s 6ms/step - loss: 3.5614 - val_loss: 5.4574
Epoch 331/10000
5/5 [==============================] - 0s 6ms/step - loss: 3.7062 - val_loss: 3.7255
Epoch 

5/5 [==============================] - 0s 7ms/step - loss: 2.9404 - val_loss: 6.1652
Epoch 485/10000
5/5 [==============================] - 0s 7ms/step - loss: 3.3066 - val_loss: 6.0419
Epoch 486/10000
5/5 [==============================] - 0s 7ms/step - loss: 2.2736 - val_loss: 7.4122
Epoch 487/10000
5/5 [==============================] - 0s 7ms/step - loss: 2.7253 - val_loss: 7.9875
Epoch 488/10000
5/5 [==============================] - 0s 7ms/step - loss: 2.4965 - val_loss: 5.9132
Epoch 489/10000
5/5 [==============================] - 0s 7ms/step - loss: 2.4038 - val_loss: 8.1951
Epoch 490/10000
5/5 [==============================] - 0s 7ms/step - loss: 2.3650 - val_loss: 9.6586
Epoch 491/10000
5/5 [==============================] - 0s 6ms/step - loss: 2.4962 - val_loss: 9.2071
Epoch 492/10000
5/5 [==============================] - 0s 6ms/step - loss: 2.6416 - val_loss: 7.1713
Epoch 493/10000
5/5 [==============================] - 0s 6ms/step - loss: 2.1806 - val_loss: 8.8999
Epoch 

5/5 [==============================] - 0s 7ms/step - loss: 1.2854 - val_loss: 4.7164
Epoch 647/10000
5/5 [==============================] - 0s 5ms/step - loss: 1.7402 - val_loss: 4.6348
Epoch 648/10000
5/5 [==============================] - 0s 6ms/step - loss: 1.7487 - val_loss: 6.5318
Epoch 649/10000
5/5 [==============================] - 0s 5ms/step - loss: 1.7678 - val_loss: 6.8126
Epoch 650/10000
5/5 [==============================] - 0s 6ms/step - loss: 1.7580 - val_loss: 5.9381
Epoch 651/10000
5/5 [==============================] - 0s 6ms/step - loss: 1.5740 - val_loss: 4.7796
Epoch 652/10000
5/5 [==============================] - 0s 7ms/step - loss: 2.0748 - val_loss: 5.0839
Epoch 653/10000
5/5 [==============================] - 0s 6ms/step - loss: 1.5660 - val_loss: 6.0346
Epoch 654/10000
5/5 [==============================] - 0s 6ms/step - loss: 1.4663 - val_loss: 6.5540
Epoch 655/10000
5/5 [==============================] - 0s 6ms/step - loss: 1.5915 - val_loss: 4.4564
Epoch 

5/5 [==============================] - 0s 6ms/step - loss: 0.9184 - val_loss: 4.3564
Epoch 809/10000
5/5 [==============================] - 0s 6ms/step - loss: 1.0054 - val_loss: 4.7861
Epoch 810/10000
5/5 [==============================] - 0s 6ms/step - loss: 0.9119 - val_loss: 3.9291
Epoch 811/10000
5/5 [==============================] - 0s 6ms/step - loss: 1.2374 - val_loss: 3.5838
Epoch 812/10000
5/5 [==============================] - 0s 6ms/step - loss: 1.3786 - val_loss: 3.9273
Epoch 813/10000
5/5 [==============================] - 0s 6ms/step - loss: 1.0852 - val_loss: 5.1257
Epoch 814/10000
5/5 [==============================] - 0s 6ms/step - loss: 0.9111 - val_loss: 4.5025
Epoch 815/10000
5/5 [==============================] - 0s 6ms/step - loss: 1.0242 - val_loss: 4.8258
Epoch 816/10000
5/5 [==============================] - 0s 6ms/step - loss: 1.0235 - val_loss: 5.4702
Epoch 817/10000
5/5 [==============================] - 0s 7ms/step - loss: 0.9867 - val_loss: 5.0523
Epoch 

5/5 [==============================] - 0s 6ms/step - loss: 0.7629 - val_loss: 4.9954
Epoch 971/10000
5/5 [==============================] - 0s 6ms/step - loss: 0.7745 - val_loss: 4.5991
Epoch 972/10000
5/5 [==============================] - 0s 6ms/step - loss: 0.6871 - val_loss: 4.6895
Epoch 973/10000
5/5 [==============================] - 0s 6ms/step - loss: 0.6177 - val_loss: 5.1097
Epoch 974/10000
5/5 [==============================] - 0s 6ms/step - loss: 0.8705 - val_loss: 5.2033
Epoch 975/10000
5/5 [==============================] - 0s 6ms/step - loss: 0.9929 - val_loss: 5.0159
Epoch 976/10000
5/5 [==============================] - 0s 6ms/step - loss: 0.7092 - val_loss: 5.9600
Epoch 977/10000
5/5 [==============================] - 0s 6ms/step - loss: 0.6807 - val_loss: 4.4909
Epoch 978/10000
5/5 [==============================] - 0s 6ms/step - loss: 0.7086 - val_loss: 6.3260
Epoch 979/10000
5/5 [==============================] - 0s 6ms/step - loss: 0.6262 - val_loss: 4.9531
Epoch 

Epoch 1051/10000
5/5 [==============================] - 0s 6ms/step - loss: 0.6961 - val_loss: 5.8216
Epoch 1052/10000
5/5 [==============================] - 0s 7ms/step - loss: 0.7267 - val_loss: 8.0474
Epoch 1053/10000
5/5 [==============================] - 0s 7ms/step - loss: 0.6331 - val_loss: 5.9910
Epoch 1054/10000
5/5 [==============================] - 0s 7ms/step - loss: 0.7004 - val_loss: 6.2604
Epoch 1055/10000
5/5 [==============================] - 0s 6ms/step - loss: 0.8152 - val_loss: 5.1709
Epoch 1056/10000
5/5 [==============================] - 0s 6ms/step - loss: 0.5678 - val_loss: 6.1682
Epoch 1057/10000
5/5 [==============================] - 0s 6ms/step - loss: 0.7975 - val_loss: 4.7512
Epoch 1058/10000
5/5 [==============================] - 0s 6ms/step - loss: 0.6875 - val_loss: 6.5581
Epoch 1059/10000
5/5 [==============================] - 0s 7ms/step - loss: 0.6219 - val_loss: 6.7865
Epoch 1060/10000
5/5 [==============================] - 0s 6ms/step - loss: 0.6816

5/5 [==============================] - 0s 6ms/step - loss: 0.4945 - val_loss: 5.4902
Epoch 1212/10000
5/5 [==============================] - 0s 6ms/step - loss: 0.5482 - val_loss: 4.6730
Epoch 1213/10000
5/5 [==============================] - 0s 6ms/step - loss: 0.5381 - val_loss: 4.5836
Epoch 1214/10000
5/5 [==============================] - 0s 6ms/step - loss: 0.6263 - val_loss: 5.1027
Epoch 1215/10000
5/5 [==============================] - 0s 6ms/step - loss: 0.5015 - val_loss: 5.4914
Epoch 1216/10000
5/5 [==============================] - 0s 6ms/step - loss: 0.5298 - val_loss: 5.8562
Epoch 1217/10000
5/5 [==============================] - 0s 6ms/step - loss: 0.5619 - val_loss: 5.0148
Epoch 1218/10000
5/5 [==============================] - 0s 6ms/step - loss: 0.6488 - val_loss: 5.1220
Epoch 1219/10000
5/5 [==============================] - 0s 6ms/step - loss: 0.4848 - val_loss: 4.9843
Epoch 1220/10000
5/5 [==============================] - 0s 6ms/step - loss: 0.6158 - val_loss: 5.21

In [36]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input.iloc[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [37]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                            test_input, 
                                            test_target, 
                                            train_split)
pd.set_option('display.max_columns', None)
df_results.T

1/1 [==============================] - 0s 15ms/step


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Month,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11,Month-12,Month-13,Month-14,Month-15,Month-16,Month-17,Month-18,Month-19,Month-20,Month-21,Month-22,Month-23,Month-24,Month-25,Month-26,Month-27,Month-28,Month-29,Month-30
Prediction,12.075612,11.794294,12.725159,13.369139,13.474021,15.497648,-5.337283,-3.581975,2.114866,7.417327,6.852262,10.898136,15.082037,12.370364,16.457823,15.02827,14.124835,18.845181,1.78677,1.825071,4.085227,4.000364,5.643169,5.922133,7.649185,16.481382,17.689068,17.943319,17.191544,17.607769
Target,12.399,11.268,8.741,12.358,11.876,11.713,11.449,7.805,7.726,8.516,8.503,10.175,11.775,9.724,10.069,13.93,11.918,11.757,9.691,7.378,6.97,7.612,8.442,8.835,11.251,11.342,13.11,13.195,10.053,12.274
Error,0.323388,0.526295,3.984159,1.011139,1.598021,3.784648,16.786283,11.386975,5.611134,1.098672,1.650738,0.723136,3.307037,2.646364,6.388823,1.098269,2.206835,7.088181,7.90423,5.552928,2.884773,3.611636,2.798831,2.912867,3.601815,5.139382,4.579068,4.74832,7.138543,5.333769


In [38]:
display(mae)
display(mape)

4.247542

0.4304326

In [39]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input.iloc[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [40]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             test_input, 
                                             test_target, 
                                             train_split)
display(errors)
display(mae)
display(mape)

1/1 [==============================] - 0s 15ms/step
Ano-0: |Prediction[[97.29921]] - Target[122.52900000000001]| =  Error: [[25.22979]]; MAPE:[[0.20590872]]
1/1 [==============================] - 0s 15ms/step
Ano-0: |Prediction[[115.17125]] - Target[118.101]| =  Error: [[2.9297485]]; MAPE:[[0.02480714]]
1/1 [==============================] - 0s 15ms/step
Ano-5: |Prediction[[94.56227]] - Target[71.22500000000001]| =  Error: [[23.337273]]; MAPE:[[0.32765564]]


[array([[25.22979]], dtype=float32),
 array([[2.9297485]], dtype=float32),
 array([[23.337273]], dtype=float32)]

17.165604

0.18612383